## Task 4

Using Google's universal encoder, calculate the similarity and replace the similar chunk in the sentence.

In [6]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)

module large_param loaded


In [3]:
import spacy
import pickle
activated = spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [48]:
# Calculate the similarity between x and y
def semantic_similarity(x, y):
    features = embed([x, y])
    corr = np.inner(features, features)
    return corr[1][0]

def print_step(step, first, second):
    print("======== step ", step, " ==========")
    print(first)
    print(second)
    print("score : ", str(semantic_similarity(first, second)), "\n")

# Replace the from_doc until similar to the to_doc
def convert(from_doc, to_doc):
    step = 0
    
    first = from_doc
    second = to_doc
    while True:
        print_step(step, first, second)
        
        if semantic_similarity(first, second) > 1 - 1e-3:
            break
            
        doc_f = nlp(first.strip())
        doc_s = nlp(second.strip())
        mx = 0
        fid, sid = None, None
        for token_f in doc_f:
            for token_s in doc_s:
                if token_f.tag_ != token_s.tag_:
                    continue
                first_p = doc_f[token_f.left_edge.i : token_f.right_edge.i+1].text.strip()
                second_p = doc_s[token_s.left_edge.i : token_s.right_edge.i+1].text.strip()
                score = semantic_similarity(first_p, second_p) 
                if score < 1 - 1e-3 and score > mx:
                    mx = score
                    fid = first_p
                    sid = second_p
        
        first = first.replace(fid, sid)
        step += 1

x = 'Understand customer needs'
y = 'Capture business requirements'

convert(x, y)

======== step  0  ==========
Understand customer needs
Capture business requirements
score :  0.3856976 

======== step  1  ==========
Understand business requirements
Capture business requirements
score :  0.64280045 

======== step  2  ==========
Capture business requirements
Capture business requirements
score :  0.99999994 

